In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
!nvidia-smi

Tue Mar 30 07:23:40 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    30W /  70W |    222MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras import optimizers
import nltk
import json
import ast
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
import os
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.layers.normalization import BatchNormalization
from collections import Counter

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_img_path='/content/drive/MyDrive/Datasets/FYPROJECT/train/'
train_captions_path='/content/drive/MyDrive/Datasets/FYPROJECT/train_data.txt'
with open(train_captions_path, "r") as data:
    train_data = ast.literal_eval(data.read())

counter = Counter()
img_list=[]
path_list=[]
cap_list=[]
class_list=[]
unique_words=[]
dictall={}
id_list=[]
word_freq=Counter()

ids = train_data.keys()

for i, id in enumerate(ids):
    img_class=str(train_data[id][0])
    
    image_name=str(train_data[id][0]+'_'+train_data[id][1])
   
    complete_path=train_img_path+'instagram_dataset'+'/'+train_data[id][0]+'/'+train_data[id][1]

    caption=train_data[id][2]
    
    tokens = nltk.tokenize.word_tokenize(caption.lower())
    word_freq.update(tokens)
    if len(tokens)>=1:
        id_list.append(image_name) 
        path_list.append(complete_path)
        class_list.append(img_class)
        cap_part=caption.split()
        sub_cap=[]
        for cap in cap_part:

            sub_cap.append(str(cap))
            if cap not in unique_words:
                unique_words.append(cap)
        cap_list.append(sub_cap)
            
 
        
Data={'Id':id_list,
      'Path':path_list,
      'image_class':class_list,
      'Genre':cap_list
}

for word in unique_words:
    word_list=[]
    
    #print(word)
    for cap in cap_list:
        #print(cap)
        if word in cap:
            word_list.append(1)
            
        else:
            word_list.append(0)
    Data[word]=word_list
    #print(Data[word])
from pandas import DataFrame 
col=['Id','Path','image_class','Genre']
for word in unique_words:
    col.append(str(word))
train = DataFrame (Data, columns = col)





In [ ]:
train.head()

,Id,Path,image_class,Genre,goodtimes,people,latepost,motivation,fit,fitnessaddict,funny,instagood,dinner,home,boy,chef,fitness,ab,workout,instamood,instalike,instadaily,sunnyday,leg,gay,instagay,throwback,boyfriend,love,mylove,tbt,throwbackthursday,likeforlike,followme,new,girl,selfie,hair,beauty,style,...,season,xoxo,braid,tshirt,cocktail,rap,banana,dessertporn,familytime,perfection,instafun,artistic,myworld,fitnessmodel,outdoor,trend,statigram,selfienation,following,hiking,hashtags,class,grunge,goingout,horizon,summertime,bloom,indie,walking,feeling,mother,hipster,wake,gain,purse,mothernature,scene,doubletap,swagger,wcw
0,goodtimes_image_2320.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,goodtimes,"[goodtimes, people, latepost]",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,funny_image_608.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,funny,"[motivation, fit, fitnessaddict, funny, instag...",0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,boy_image_303.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,boy,"[dinner, home, boy, chef]",0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,funny_image_760.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,funny,"[fitness, ab, funny, workout, instamood, insta...",0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,goodtimes_image_608.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,goodtimes,"[sunnyday, leg, gay, instagay, goodtimes]",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
new_train=train[['Id','Path','image_class','Genre']]
df=pd.get_dummies(new_train['image_class'],drop_first=True)
train_df=pd.concat([new_train,df],axis=1)
train_df.head()

,Id,Path,image_class,Genre,awesome,beach,beauty,black,bored,boy,boyfriend,cake,colorful,cool,dog,drunk,eyes,family,fashion,friday,friends,funny,girls,goodtimes,green,hair,heart,home,igdaily,instafood,justinbieber,loveyou,makeup,model,new,nike,old,red,school,sea,shoes,smile,snow,sun,tattoo,throwback,tired,tree,truth,white,yellow,yum,yummy
0,goodtimes_image_2320.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,goodtimes,"[goodtimes, people, latepost]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,funny_image_608.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,funny,"[motivation, fit, fitnessaddict, funny, instag...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,boy_image_303.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,boy,"[dinner, home, boy, chef]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,funny_image_760.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,funny,"[fitness, ab, funny, workout, instamood, insta...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,goodtimes_image_608.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/trai...,goodtimes,"[sunnyday, leg, gay, instagay, goodtimes]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_df.shape

(4000, 53)

In [ ]:
train_image = []
for i in tqdm(range(train_df.shape[0]),position=0, leave=True):
    img = image.load_img(train_df['Path'][i],target_size=(227,227,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X_train = np.array(train_image)
    
y_train = np.array(train_df.drop(['Id', 'Path','image_class','Genre'],axis=1))

100%|██████████| 4000/4000 [20:56<00:00,  3.18it/s]


In [ ]:
test_captions_path='/content/drive/MyDrive/Datasets/FYPROJECT/vald_data.txt'
test_img_path='/content/drive/MyDrive/Datasets/FYPROJECT/vald/'
with open(test_captions_path, "r") as data:
    test_data = ast.literal_eval(data.read())

counter = Counter()
img_list=[]
path_list=[]
cap_list=[]
class_list=[]
dictall={}
id_list=[]

ids = test_data.keys()

for i, id in enumerate(ids):
    img_class=str(test_data[id][0])
    
    image_name=str(test_data[id][0]+'_'+test_data[id][1])
   
    complete_path=test_img_path+'instagram_dataset'+'/'+test_data[id][0]+'/'+test_data[id][1]
   
    caption=test_data[id][2]
    tokens = nltk.tokenize.word_tokenize(caption.lower())
    word_freq.update(tokens)
    if len(tokens)>=1:
        id_list.append(image_name) 
        path_list.append(complete_path)
        cap_part=caption.split()
        class_list.append(img_class)
        sub_cap=[]
        for cap in cap_part:

            sub_cap.append(str(cap))
            #if cap not in unique_words:
                #unique_words.append(cap)
        cap_list.append(sub_cap)
    
  
    
valData={'Id':id_list,
      'Path':path_list,
      'image_class':class_list,
      'Genre':cap_list
         
    
}
for word in unique_words:
    word_list=[]
    
    #print(word)
    for cap in cap_list:
        #print(cap)
        if word in cap:
            word_list.append(1)
            
        else:
            word_list.append(0)
    valData[word]=word_list
    #print(valData[word])
from pandas import DataFrame 
col=['Id','Path','image_class','Genre']
for word in unique_words:
    col.append(str(word))
test= DataFrame (valData, columns = col)





In [ ]:
new_test=test[['Id','Path','image_class','Genre']]
df=pd.get_dummies(new_test['image_class'],drop_first=True)
test_df=pd.concat([new_test,df],axis=1)
test_df.head()

,Id,Path,image_class,Genre,awesome,beach,beauty,black,bored,boy,boyfriend,cake,colorful,cool,dog,drunk,eyes,family,fashion,friday,friends,funny,girls,goodtimes,green,hair,heart,home,igdaily,instafood,justinbieber,loveyou,makeup,model,new,nike,old,red,school,sea,shoes,smile,snow,sun,tattoo,throwback,tired,tree,truth,white,yellow,yum,yummy
0,white_image_334.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/vald...,white,"[snow, winter, white]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,awesome_image_620.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/vald...,awesome,"[beach, awesome]",1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,tree_image_1844.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/vald...,tree,"[bnw, monochrome, vscocam, vsco, streetphotogr...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,instafood_image_2424.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/vald...,instafood,"[cake, pink, spring, foodpic, instafood]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,funny_image_2698.jpg,/content/drive/MyDrive/Datasets/FYPROJECT/vald...,funny,"[true, funny, lol, humor, haha, lmao, comedy, ...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_image = []
for i in tqdm(range(test_df.shape[0]),position=0, leave=True):
    img = image.load_img(test_df['Path'][i],target_size=(227,227,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X_test = np.array(test_image)
    
y_test = np.array(test_df.drop(['Id', 'Path','image_class','Genre'],axis=1))

100%|██████████| 500/500 [02:32<00:00,  3.28it/s]


In [ ]:
from keras.applications import VGG19 #For Transfer Learning
base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (227,227,3), classes = y_test.shape[1])
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 
model.add(Dense(1024,activation=('relu'),input_dim=512))
model.add(Dense(512,activation=('relu'))) 
model.add(Dense(256,activation=('relu'))) 
model.add(Dropout(.3))
model.add(Dense(128,activation=('relu')))
#model.add(Dropout(.2))
model.add(Dense(49,activation=('softmax'))) 

#Checking the final model summary
model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________

In [ ]:

#Initializing the hyperparameters
batch_size= 100
epochs=50
learn_rate=.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
#adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test), batch_size=100)

Epoch 1/25
40/40 [==============================] - 141s 2s/step - loss: 3.8859 - accuracy: 0.0183 - val_loss: 3.8083 - val_accuracy: 0.0360
Epoch 2/25
40/40 [==============================] - 79s 2s/step - loss: 3.8052 - accuracy: 0.0226 - val_loss: 3.8035 - val_accuracy: 0.0200
Epoch 3/25
40/40 [==============================] - 79s 2s/step - loss: 3.8048 - accuracy: 0.0362 - val_loss: 3.7878 - val_accuracy: 0.0380
Epoch 4/25
40/40 [==============================] - 79s 2s/step - loss: 3.7740 - accuracy: 0.0476 - val_loss: 3.7647 - val_accuracy: 0.0420
Epoch 5/25
40/40 [==============================] - 79s 2s/step - loss: 3.7215 - accuracy: 0.0594 - val_loss: 3.7361 - val_accuracy: 0.0520
Epoch 6/25
40/40 [==============================] - 79s 2s/step - loss: 3.6233 - accuracy: 0.0757 - val_loss: 3.6756 - val_accuracy: 0.0960
Epoch 7/25
40/40 [==============================] - 79s 2s/step - loss: 3.5046 - accuracy: 0.1063 - val_loss: 3.6237 - val_accuracy: 0.0900
Epoch 8/25
40/40 [=

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/MyDrive/Datasets/FYPROJECT/hybrid_recommender/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('/content/drive/MyDrive/Datasets/FYPROJECT/hybrid_recommender/model.h5')
print("Saved model to disk")

Saved model to disk


In [ ]:
test_captions_path='/content/drive/MyDrive/Datasets/FYPROJECT/test_data.txt'
test_img_path='/content/drive/MyDrive/Datasets/FYPROJECT/test/'
with open(test_captions_path, "r") as data:
    test_data = ast.literal_eval(data.read())
counter = Counter()
all_test_img_paths=[]
all_test_captions=[]
ids = test_data.keys()
save_in_file = open('/content/drive/MyDrive/Datasets/FYPROJECT/hybrid_recommender/testresult_1.txt', "w")
total_f1=0
val_data_length=0
for i, id in enumerate(ids):
    val_data_length=val_data_length+1
    pred=[]
    image_name=test_data[id][1]
    print("\nimage name : {}".format(test_data[id][1]))
    complete_path=test_img_path+'instagram_dataset'+'/'+test_data[id][0]+'/'+test_data[id][1]
    print("\nimage path : {}".format(complete_path))
    #caption='<start> '+test_data[id][2]+' <end>'
    caption=test_data[id][2]
    img = image.load_img(complete_path,target_size=(227,227,3))
    img = image.img_to_array(img)
    img = img/255
    classes = np.array(train_df.columns[4:])
    #proba = model.predict(img.reshape(1,227,227,3))
    proba=[]
    proba = model.predict(img.reshape(1,227,227,3))
    top_3 = np.argsort(proba[0])[:-4:-1]
    for i in range(3):
        #print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
        pred.append(classes[top_3[i]])
    class_name=pred[0]
    json_file = open('/content/drive/MyDrive/Datasets/FYPROJECT/hybrid_recommender/model_{}.json'.format(class_name), 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("/content/drive/MyDrive/Datasets/FYPROJECT/hybrid_recommender/model_{}.h5".format(class_name))
    pred=[]
    proba=[]
    proba = loaded_model.predict(img.reshape(1,227,227,3))
    top_3 = np.argsort(proba[0])[:-4:-1]
    for i in range(3):
        print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
        pred.append(str(classes[top_3[i]]))
    save_in_file.write(complete_path)
    save_in_file.write(';')
    #text='original caption : '+str(caption)
    act=caption.split()
    res = len([key for key, val in enumerate(act) if val in set(pred)])
    precision=res/len(pred)
    recall=res/len(act)
    if(precision==0):
      f1=0
    elif(recall==0):
      f1=0
    else:
      f1=2*precision*recall/precision+recall
    total_f1=total_f1+f1
    print(f1)
    text=str(caption.split())
    save_in_file.write(text)
    save_in_file.write(';')
    #text='predicted caption : '+str(pred)
    text=str(pred)
    save_in_file.write(text)
    save_in_file.write('\n')
    
total_f1=total_f1/val_data_length
print(total_f1)            
save_in_file.close()

cool (nan)
colorful (nan)
cake (nan)
0

image name : image_2412.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram_dataset/girls/image_2412.jpg
cool (nan)
colorful (nan)
cake (nan)
0

image name : image_2216.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram_dataset/model/image_2216.jpg
cool (nan)
colorful (nan)
cake (nan)
0

image name : image_608.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram_dataset/home/image_608.jpg
beach (1.0)
cool (0.0)
colorful (0.0)
0

image name : image_6138.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram_dataset/justinbieber/image_6138.jpg
awesome (0.852)
beach (0.0694)
beauty (0.0162)
0

image name : image_4408.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram_dataset/boyfriend/image_4408.jpg
awesome (1.0)
cool (0.0)
colorful (0.0)
0

image name : image_8367.jpg

image path : /content/drive/MyDrive/Datasets/FYPROJECT/test/instagram